In [1]:
# import tensorflow as tf
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import keras.layers as layers
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler


import numpy as np
import PIL

tf.config.list_physical_devices('GPU')

[]

---
### 데이터
---

In [22]:
(X_train,y_train),(X_test,y_test) = cifar10.load_data()

In [23]:
X_train = X_train.astype(float)/255
X_test = X_test.astype(float)/255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [24]:
X_train.shape

(50000, 32, 32, 3)

In [25]:
y_train.shape

(50000, 10)

---
### 모델 정의
---

In [26]:
class LenetLike(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.conv_layer1 = tf.keras.Sequential([
            layers.Conv2D(filters=6,kernel_size=5,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer1 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.conv_layer2 = tf.keras.Sequential([
            layers.Conv2D(filters=16,kernel_size=5,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer2 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.flatten = layers.Flatten() 

        self.fc_layer1 = tf.keras.Sequential([
        layers.Dense(512),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.fc_layer2 = tf.keras.Sequential([
        layers.Dense(512),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.output_layer = tf.keras.Sequential([
        layers.Dense(10),
        layers.Softmax()
        ])

    def call(self,x):
        x = self.conv_layer1(x)
        x = self.sub_sampling_layer1(x)
        x = self.conv_layer2(x)
        x = self.sub_sampling_layer2(x)
        x = self.flatten(x)
        x = self.fc_layer1(x)
        x = self.fc_layer2(x)
        x = self.output_layer(x)
        return x

In [27]:
model = LenetLike()
model.build(input_shape=(None,32,32,3))
model.summary()

Model: "lenet_like_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_27 (Sequential)  (None, 32, 32, 6)         480       
                                                                 
 max_pooling2d_12 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 sequential_28 (Sequential)  (None, 16, 16, 16)        2480      
                                                                 
 max_pooling2d_13 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         multiple                  0         
                                                                 
 sequential_29 (Sequential)  (None, 512)              

In [28]:
sample = np.random.rand(32,32,32,3)
model.predict(sample).shape

1/1 [==============================] - 0s 74ms/step


(32, 10)

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

stop_callback = EarlyStopping(monitor='val_accuracy',patience=5)

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

lr_callbacks = LearningRateScheduler(scheduler)

model.fit(x=X_train,y=y_train,
          batch_size=128,
          epochs=100,
          validation_data=(X_test,y_test),
          callbacks=[stop_callback])

Epoch 1/100
391/391 [==============================] - 25s 61ms/step - loss: 2.5989 - accuracy: 0.1306 - val_loss: 2.1603 - val_accuracy: 0.2487
Epoch 2/100
391/391 [==============================] - 24s 62ms/step - loss: 2.2428 - accuracy: 0.1891 - val_loss: 2.0305 - val_accuracy: 0.3038
Epoch 3/100
391/391 [==============================] - 25s 65ms/step - loss: 2.1354 - accuracy: 0.2294 - val_loss: 1.9535 - val_accuracy: 0.3291
Epoch 4/100
391/391 [==============================] - 25s 63ms/step - loss: 2.0587 - accuracy: 0.2567 - val_loss: 1.8886 - val_accuracy: 0.3492
Epoch 5/100
391/391 [==============================] - 28s 72ms/step - loss: 1.9905 - accuracy: 0.2831 - val_loss: 1.8289 - val_accuracy: 0.3691
Epoch 6/100
391/391 [==============================] - 31s 78ms/step - loss: 1.9439 - accuracy: 0.3003 - val_loss: 1.7829 - val_accuracy: 0.3797
Epoch 7/100
391/391 [==============================] - 29s 74ms/step - loss: 1.8947 - accuracy: 0.3182 - val_loss: 1.7400 - val_ac

In [ ]:
prediction = model.predict(X_test)

In [ ]:
(prediction.argmax(-1) == y_test.argmax(-1)).sum()